In [ ]:
# default_exp data
# default_cls_lvl 3

In [ ]:
#export
import pandas as pd
import numpy as np

from task_substitution.core import *

# Dataset

> Class that would represent dataset.

In [ ]:
#hide
from nbdev.showdoc import *

### Dataset Class

In [ ]:
#export
class Dataset:
    def __init__(self, df, **kwargs):
        self.df = df
        self.target_fld = kwargs['target_fld']
        self.ignore_flds = kwargs['ignore_flds']
        self.cat_flds = kwargs['cat_flds']
        
    @property
    def target(self):
        return self.data[self.target_fld]
    
    def remove_ignore_flds(self)->pd.DataFrame:
        if self.ignore_flds is not None:
            self.df = _ignore_flds(self.df, self.ignore_flds)
        return self.df
    
    def preprocess_categorical(self)->pd.DataFrame:
        cats = self.cat_flds if self.cat_flds else self.df.select_dtypes(include=['object']).columns
        for c, v in self.df.loc[:, cats].items():
            self.df.loc[:, c] = _preprocess_categorical(v)
        return self.df
    
    def preprocess(self):
        self.df = self.remove_ignore_flds()
        self.df = self.preprocess_categorical()
        
        return self.df
    
    @classmethod
    def split_train_test_by_null(cls, df:pd.DataFrame, target_fld:str)->(pd.DataFrame, pd.DataFrame):
        train, test = _split_by_null(df, target_fld)
        return train, test
    
    @classmethod
    def split_train_test(cls, df:pd.DataFrame, split_args:dict)->(pd.DataFrame, pd.DataFrame):
        """
        Splits the dataframe into train and test sets.
        """
        train, test = _split_train_test(df, split_args)
        return train, test

In [ ]:
import string
example_df = pd.DataFrame({'c1': np.random.rand(10, ),
                           'c2': [string.ascii_lowercase[np.random.randint(low=0, high=10)] for i in range(10)],
                           'c3': np.random.permutation([np.nan] + list(np.random.rand(9, )))
                          })
example_df

,c1,c2,c3
0,0.986508,a,0.135839
1,0.738145,c,0.245510
2,0.041694,d,0.124175
3,0.358501,i,0.905612
4,0.822294,h,0.296354
5,0.851814,d,0.111530
6,0.068141,g,0.981517
7,0.194940,e,0.752494
8,0.337356,b,0.319017
9,0.656170,e,NaN


In [ ]:
data = Dataset(example_df, target_fld='c3', cat_flds=['c2'], ignore_flds=None)
new_example_df = data.preprocess(); new_example_df

assert len(new_example_df.loc[:, ['c2']].select_dtypes(include=['object']).columns) == 0